# Data Analysis and Machine Learning on Crime Data

This notebook performs data analysis and machine learning on crime data. It includes data import, visualization, preprocessing, and model training.

## Install Required Libraries

In [44]:
try :
    import pandas as pd
except Exception as e:
    !pip install pandas > /dev/null
try:
    import plotly
except Exception as e:
    !pip install plotly > /dev/null
try:
    import category_encoders
except Exception as e:
    !pip install category_encoders > /dev/null
try:
    import sklearn
except Exception as e:
    !pip install sklearn > /dev/null
try:
    import matplotlib
except Exception as e:
    !pip install matplotlib > /dev/null

## Import Libraries

In [45]:
import pandas as pd
import os
import plotly.express as px
from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings
from collections import Counter

## Define Functions

In [46]:
def import_csv(file_path):
    """Import a CSV file and return a pandas DataFrame."""
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except pd.errors.EmptyDataError:
        print("Error: File is empty.")
    except pd.errors.ParserError:
        print("Error: Parsing issue in the file.")
    except Exception as e:
        print(f"Unexpected error: {e}")

def create_file_path(*path_segments):
    """Create a file path from path segments."""
    return os.path.join(*path_segments)

def graphpoint(datas, x, y, color):
    """Display a scatter plot."""
    fig = px.scatter(datas, x=x, y=y, color=color)
    fig.show()

def graphbox(datas):
    """Display a box plot."""
    datas.plot.box()

def graphhistogram(datas, x, color):
    """Return a histogram plot."""
    return px.histogram(datas, x=x, color=color)

def datasprint(datas, describe, head):
    """Print data description and head."""
    if describe:
        print(datas.describe())
    if head:
        print(datas.head())

## Load CSV Files

In [47]:
directory = './DataSet'
file_path = create_file_path(directory, "test.csv")
file_path2 = create_file_path(directory, "train.csv")

df_test = import_csv(file_path)
df_train = import_csv(file_path2)

if df_test is None or df_train is None:
    raise Exception("One of the dataframes is empty")


## Display Data

In [48]:
print(f"{file_path} :")
datasprint(df_test, describe=True, head=True)

./DataSet\test.csv :
                  Id              X              Y
count  884262.000000  884262.000000  884262.000000
mean   442130.500000    -122.422693      37.771476
std    255264.596205       0.030985       0.484824
min         0.000000    -122.513642      37.707879
25%    221065.250000    -122.433069      37.752374
50%    442130.500000    -122.416517      37.775421
75%    663195.750000    -122.406959      37.784353
max    884261.000000    -120.500000      90.000000
   Id                Dates DayOfWeek PdDistrict                   Address  \
0   0  2015-05-10 23:59:00    Sunday    BAYVIEW   2000 Block of THOMAS AV   
1   1  2015-05-10 23:51:00    Sunday    BAYVIEW        3RD ST / REVERE AV   
2   2  2015-05-10 23:50:00    Sunday   NORTHERN    2000 Block of GOUGH ST   
3   3  2015-05-10 23:45:00    Sunday  INGLESIDE  4700 Block of MISSION ST   
4   4  2015-05-10 23:45:00    Sunday  INGLESIDE  4700 Block of MISSION ST   

            X          Y  
0 -122.399588  37.735051  
1 -

In [49]:
print(f"{file_path2} :")
datasprint(df_train, describe=True, head=True)

./DataSet\train.csv :
                   X              Y
count  878049.000000  878049.000000
mean     -122.422616      37.771020
std         0.030354       0.456893
min      -122.513642      37.707879
25%      -122.432952      37.752427
50%      -122.416420      37.775421
75%      -122.406959      37.784369
max      -120.500000      90.000000
                 Dates        Category                      Descript  \
0  2015-05-13 23:53:00        WARRANTS                WARRANT ARREST   
1  2015-05-13 23:53:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
2  2015-05-13 23:33:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
3  2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   
4  2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   

   DayOfWeek PdDistrict      Resolution                    Address  \
0  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
1  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
2  Wednesda

## Étude des données

> Regarder la donnée en fonction de la résolution

Nous allons regarder les données en fonction de la résolution pour voir si nous pouvons trouver des corrélations. Nous allons afficher des histogrammes pour chaque colonne de données en fonction de la résolution. Cela nous permettra de voir si certaines colonnes ont un impact sur la résolution.

Nous avons donc constater le nombre de résolutions par crime et très élévé. Nous avons donc décidé de regrouper les résolutions en 4 catégories:
- Arrestation / Poursuites : ARREST, BOOKED, ARREST, CITED, JUVENILE CITED, JUVENILE BOOKED, PROSECUTED FOR LESSER OFFENSE, PROSECUTED BY OUTSIDE AGENCY
- Absence de poursuites / Refus / Cas particuliers : COMPLAINANT REFUSES TO PROSECUTE, DISTRICT ATTORNEY REFUSES TO PROSECUTE, NOT PROSECUTED, JUVENILE ADMONISHED, JUVENILE DIVERTED, CLEARED-CONTACT JUVENILE FOR MORE INFO, PSYCHOPATHIC CASE, EXCEPTIONAL CLEARANCE
- Localisation / État de la personne : LOCATED, UNFOUNDED
- Indépendant (NONE) : NONE

In [50]:
# df_train_columns: list = df_train.columns.tolist()
# df_train_columns.remove('Category')
# df_train_columns.remove('Descript')
# df_train_columns.remove('Resolution')
# 
# for column in df_train_columns:
#     graphhistogram(df_train, column, 'Resolution').show()

## Data Preprocessing

In [51]:
# Categorize 'Resolution' into broader categories
df_train['Categorie'] = ''
df_train.loc[df_train['Resolution'].isin(['ARREST, BOOKED', 'ARREST, CITED', 'JUVENILE CITED', 'JUVENILE BOOKED', 'PROSECUTED FOR LESSER OFFENSE', 'PROSECUTED BY OUTSIDE AGENCY']), 'Categorie'] = 'Arrestation / Poursuites'
df_train.loc[df_train['Resolution'].isin(['COMPLAINANT REFUSES TO PROSECUTE', 'DISTRICT ATTORNEY REFUSES TO PROSECUTE', 'NOT PROSECUTED', 'JUVENILE ADMONISHED', 'JUVENILE DIVERTED', 'CLEARED-CONTACT JUVENILE FOR MORE INFO', 'PSYCHOPATHIC CASE', 'EXCEPTIONAL CLEARANCE']), 'Categorie'] = 'Absence de poursuites / Refus / Cas particuliers'
df_train.loc[df_train['Resolution'].isin(['LOCATED', 'UNFOUNDED']), 'Categorie'] = 'Localisation / État de la personne'
df_train.loc[df_train['Resolution'].isin(['NONE']), 'Categorie'] = 'Indépendant (NONE)'


## Data Visualization

> Nous allons afficher les histogrammes pour chaque colonne de données en fonction de la résolution pour visualiser l'impacte de notre choix de catégorisation.

In [52]:
# df_train_columns: list = df_train.columns.tolist()
# df_train_columns.remove('Category')
# df_train_columns.remove('Descript')
# df_train_columns.remove('Resolution')
# 
# for column in df_train_columns:
#     graphhistogram(df_train, column, 'Resolution').show()

## Feature Selection and Encoding

> Nous allons sélectionner les colonnes qui nous intéressent pour l'entraînement du modèle. Nous allons également encoder les colonnes catégorielles en valeurs numériques.

In [53]:
df_train_patch = df_train[['Dates', 'Categorie', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y']]
df_test_patch = df_test[['Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y']]

categorical_features = [col for col in df_train_patch.columns if df_train[col].dtype == 'object']
encoder = OrdinalEncoder(cols=categorical_features).fit(df_train_patch)
df_train_patch_2 = encoder.transform(df_train_patch)


## Sampling and Splitting Data

> Nous allons échantillonner les données pour réduire la taille de l'ensemble d'entraînement. Nous allons également diviser les données en ensembles d'entraînement et de test.

Nous avons décidé de prendre 25000 échantillons pour chaque catégorie. Le choix de faire un échantillonage est dû à la taille des données qui est très grande. Nous avons donc décidé de prendre un échantillon pour chaque catégorie pour réduire la taille des données. 25000 est un nombre suffisant pour avoir une bonne précision.

In [54]:
nb = 25000
df_train_patch_sample = pd.concat([
    df_train_patch_2[df_train_patch_2.Categorie == i].sample(min(nb, len(df_train_patch_2[df_train_patch_2.Categorie == i])), replace=True) 
    for i in range(4)
])

y = df_train_patch_sample.Categorie
X = df_train_patch_sample.drop(['Categorie'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Model Training and Evaluation

> Nous allons entraîner des modèles de classification sur les données d'entraînement et évaluer leur précision sur les données de test.

Nous allons entraîner trois modèles de classification différents : Decision Tree, Random Forest et K-Nearest Neighbors.

Le choix d'utiliser trois modèles différents est dû à la nature des données. Nous avons des données catégorielles et numériques. Nous avons donc décidé d'utiliser des modèles différents pour voir lequel est le plus performant.

In [55]:
# Decision Tree
clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
acctree = clf.score(X_test, y_test) * 100
print(f'Decision Tree Accuracy: {acctree:.2f}%')

# Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train, y_train)
accrf = accuracy_score(y_test, rf_classifier.predict(X_test)) * 100
print(f'Random Forest Accuracy: {accrf:.2f}%')

# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=2).fit(X_train, y_train)
accknn = accuracy_score(y_test, knn.predict(X_test)) * 100
print(f'KNN Accuracy: {accknn:.2f}%')

Decision Tree Accuracy: 56.05%
Random Forest Accuracy: 60.41%
KNN Accuracy: 44.34%


## KNeighborsClassifier - Choosing the Best K Value

> Nous allons choisir la meilleure valeur de k pour le modèle K-Nearest Neighbors en traçant l'exactitude en fonction de différentes valeurs de k.

Nous avons décidé de choisir la valeur de k qui donne la meilleure précision pour le modèle K-Nearest Neighbors.

On remarque que la précision est maximale pour k=2.

In [ ]:
k_values = range(1, 21)
accuracies = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))
plt.plot(k_values, accuracies)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Accuracy')
plt.title('Accuracy for different k values')
plt.show()

## Overall Accuracy

In [32]:
print("Overall Accuracy")
print(f'Decision Tree: {acctree:.2f}%')
print(f'Random Forest: {accrf:.2f}%')
print(f'KNN: {accknn:.2f}%')
glbacc = (acctree + accrf + accknn) / 3
print(f'Global: {glbacc:.2f}%')

Overall Accuracy
Decision Tree: 55.69%
Random Forest: 60.17%
KNN: 45.00%
Global: 53.62%


## Predict New Data

In [33]:
warnings.filterwarnings("ignore", category=FutureWarning, message=".*Downcasting object dtype arrays on .fillna.*")

new_data = {
    "Dates": ["2015-05-10 23:59:00"],
    "DayOfWeek": ["Sunday"],
    "PdDistrict": ["BAYVIEW"],
    "Address": ["2000 Block of THOMAS AV"],
    "X": [-122.39958770418998],
    "Y": [37.7350510103906],
    "Categorie": [None]
}

new_df = pd.DataFrame(new_data)
new_df_encoded = encoder.transform(new_df).drop(columns=['Categorie'])

tree_prediction = clf.predict(new_df_encoded)
rf_prediction = rf_classifier.predict(new_df_encoded)
knn_prediction = knn.predict(new_df_encoded)

## Final Prediction Using Majority Vote

In [34]:
predictions = [tuple(tree_prediction), tuple(rf_prediction), tuple(knn_prediction)]
prediction_counts = Counter(predictions)
most_common_prediction, count = prediction_counts.most_common(1)[0]

if count == 1:
    accuracies = {'tree': acctree, 'rf': accrf, 'knn': accknn}
    most_accurate_model = max(accuracies, key=accuracies.get)
    final_prediction = eval(f'{most_accurate_model}_prediction')
else:
    final_prediction = most_common_prediction

final_prediction = int(final_prediction[0])
print(f"Final Prediction: {final_prediction}")

Final Prediction: 2


## Map Prediction to Category

In [35]:
prediction_mapping = {
    0: 'Arrestation / Poursuites',
    1: 'Absence de poursuites / Refus / Cas particuliers',
    2: 'Localisation / État de la personne',
    3: 'Indépendant (NONE)'
}

final_prediction_text = prediction_mapping.get(final_prediction, "Catégorie inconnue")
print(f"Final Prediction: {final_prediction_text}")

Final Prediction: Localisation / État de la personne


## Aprés avoir fait l'analyse des données

> Non implémenté

Nous avons décidé de spliter la date dans le but de préciser les décisions prises en fonction des Date ou de l'horraire
- création de la colonne 'Date'
- création de la colonne 'Heure'
- création de la colonne 'MonthofYear'

In [43]:
df_train['Date'] = pd.to_datetime(df_train['Dates']).dt.date
df_train['Heure'] = pd.to_datetime(df_train['Dates']).dt.time
df_train['MonthofYear'] = pd.to_datetime(df_train['Date']).dt.month

print(df_train)

df_test['Date'] = pd.to_datetime(df_test['Dates']).dt.date
df_test['Heure'] = pd.to_datetime(df_test['Dates']).dt.time
df_test['MonthofYear'] = pd.to_datetime(df_test['Date']).dt.month

print(df_test)

df_train_patch = df_train[['Dates', 'Categorie', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Date', 'Heure', 'MonthofYear']]
df_test_patch = df_test[['Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Date', 'Heure', 'MonthofYear']]

categorical_features = [col for col in df_train_patch.columns if df_train[col].dtype == 'object']
encoder = OrdinalEncoder(cols=categorical_features).fit(df_train_patch)
df_train_patch_2 = encoder.transform(df_train_patch)

nb = 25000
df_train_patch_sample = pd.concat([
    df_train_patch_2[df_train_patch_2.Categorie == i].sample(min(nb, len(df_train_patch_2[df_train_patch_2.Categorie == i])), replace=True) 
    for i in range(4)
])

y = df_train_patch_sample.Categorie
X = df_train_patch_sample.drop(['Categorie'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Decision Tree
clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
acctree = clf.score(X_test, y_test) * 100
print(f'Decision Tree Accuracy: {acctree:.2f}%')

# Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train, y_train)
accrf = accuracy_score(y_test, rf_classifier.predict(X_test)) * 100
print(f'Random Forest Accuracy: {accrf:.2f}%')

# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=2).fit(X_train, y_train)
accknn = accuracy_score(y_test, knn.predict(X_test)) * 100
print(f'KNN Accuracy: {accknn:.2f}%')

print("Overall Accuracy")
print(f'Decision Tree: {acctree:.2f}%')
print(f'Random Forest: {accrf:.2f}%')
print(f'KNN: {accknn:.2f}%')
glbacc = (acctree + accrf + accknn) / 3
print(f'Global: {glbacc:.2f}%')

warnings.filterwarnings("ignore", category=FutureWarning, message=".*Downcasting object dtype arrays on .fillna.*")

new_data = {
    "Dates": ["2015-05-10 23:59:00"],
    "DayOfWeek": ["Sunday"],
    "PdDistrict": ["BAYVIEW"],
    "Address": ["2000 Block of THOMAS AV"],
    "X": [-122.39958770418998],
    "Y": [37.7350510103906],
    "Categorie": [None]
}

new_df = pd.DataFrame(new_data)
new_df_encoded = encoder.transform(new_df).drop(columns=['Categorie'])

tree_prediction = clf.predict(new_df_encoded)
rf_prediction = rf_classifier.predict(new_df_encoded)
knn_prediction = knn.predict(new_df_encoded)

predictions = [tuple(tree_prediction), tuple(rf_prediction), tuple(knn_prediction)]
prediction_counts = Counter(predictions)
most_common_prediction, count = prediction_counts.most_common(1)[0]

if count == 1:
    accuracies = {'tree': acctree, 'rf': accrf, 'knn': accknn}
    most_accurate_model = max(accuracies, key=accuracies.get)
    final_prediction = eval(f'{most_accurate_model}_prediction')
else:
    final_prediction = most_common_prediction

final_prediction = int(final_prediction[0])
print(f"Final Prediction: {final_prediction}")

prediction_mapping = {
    0: 'Arrestation / Poursuites',
    1: 'Absence de poursuites / Refus / Cas particuliers',
    2: 'Localisation / État de la personne',
    3: 'Indépendant (NONE)'
}

final_prediction_text = prediction_mapping.get(final_prediction, "Catégorie inconnue")
print(f"Final Prediction: {final_prediction_text}")

                     Dates                Category  \
0      2015-05-13 23:53:00                WARRANTS   
1      2015-05-13 23:53:00          OTHER OFFENSES   
2      2015-05-13 23:33:00          OTHER OFFENSES   
3      2015-05-13 23:30:00           LARCENY/THEFT   
4      2015-05-13 23:30:00           LARCENY/THEFT   
...                    ...                     ...   
878044 2003-01-06 00:15:00                 ROBBERY   
878045 2003-01-06 00:01:00           LARCENY/THEFT   
878046 2003-01-06 00:01:00           LARCENY/THEFT   
878047 2003-01-06 00:01:00               VANDALISM   
878048 2003-01-06 00:01:00  FORGERY/COUNTERFEITING   

                                         Descript  DayOfWeek PdDistrict  \
0                                  WARRANT ARREST  Wednesday   NORTHERN   
1                        TRAFFIC VIOLATION ARREST  Wednesday   NORTHERN   
2                        TRAFFIC VIOLATION ARREST  Wednesday   NORTHERN   
3                    GRAND THEFT FROM LOCKED AUTO  

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int32DType'>)